# MNIST Training using PyTorch

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [ ]:
import sagemaker

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial

from sagemaker.debugger import (
    Rule, ProfilerRule, rule_configs, ProfilerConfig, 
    FrameworkProfile, DetailedProfilingConfig, 
    DataloaderProfilingConfig, PythonProfilingConfig)
import time

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

In [ ]:
sagemaker.__version__

In [ ]:
inputs = f"s3://{bucket}/{prefix}"
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## Train
### Training script
The script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

## Setup fsx and use fsx for data channels and checkpoints

While the above option is easier to setup, using an FSX can be beneficial for performance when dealing with large input sizes and large model sizes. 

Please see the instructions [here](https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb), to create the FSx lustre filesystem and import the dataset from the S3 bucket to your fsx filesystem. Note that the FSX must be created in a private subnet with internet gateway to ensure that training job has access to the internet. 

In [ ]:
# # # Instructions obtained from:

# use_fsx = False

# if use_fsx:
#     from sagemaker.inputs import FileSystemInput

#     # Specify FSx Lustre file system id.
#     # file_system_id = "<your-file-system-id>"
#     file_system_id = 'fs-099a02ed242945403'

#     # Specify directory path for input data on the file system.
#     # You need to provide normalized and absolute path below.
#     # Your mount name can be provided by you when creating fsx, or generated automatically.
#     # You can find this mount_name on the FSX page in console.
#     # Example of fsx generated mount_name: "3x5lhbmv"
#     # base_path = "<your-mount-name>"
#     base_path = '/q7rzxbmv'

#     # Specify your file system type.
#     file_system_type = "FSxLustre"

#     training = FileSystemInput(file_system_id=file_system_id,
#                             file_system_type=file_system_type,
#                             directory_path=base_path + "/git-dataset",
#                             file_system_access_mode="rw")

In [ ]:
metric_definitions=[
     {'Name': 'train:Loss', 'Regex': 'Train Loss: (.*?),'},
     {'Name': 'test:Loss', 'Regex': 'Average loss: (.*?),'},
     {'Name': 'test:Accuracy', 'Regex': 'Accuracy: (.*?)%'},
]

### Model training with SMDataParallel


The training script provides the code you need for distributed data parallel (DDP) training using SMDataParallel. The training script is very similar to a PyTorch training script you might run outside of SageMaker, but modified to run with SMDataParallel. SMDataParallel's PyTorch client provides an alternative to PyTorch's native DDP. For details about how to use SMDataParallel's DDP in your native PyTorch script, see the Getting Started with SMDataParallel tutorials.

In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell.

**Instance types**

SMDataParallel supports model training on SageMaker with the following instance types only:
1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of SMDataParallel, you should use at least 2 instances, but you can also use 1 for testing this example.

In [ ]:
instance_type = 'ml.p3.16xlarge' # 'ml.p3.2xlarge'  # 'ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge', 'local_gpu'
instance_type = 'local_gpu'
instance_count = 1
use_spot_instances = False
max_wait = None
max_run = 1*60*60

In [ ]:
if instance_type =='local_gpu':
    from sagemaker.local import LocalSession
    from pathlib import Path

    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    s3_data_path = f'file://{Path.cwd()}/data'
    source_dir = f'{Path.cwd()}/train_code'
else:
    sagemaker_session = sagemaker.Session()
    s3_data_path = inputs
    source_dir = 'train_code'


**Distribution strategy**

Note that to use DDP mode, you update the the `distribution` strategy, and set it to use `smdistributed dataparallel`. 

In [ ]:
distribution = {}

flag = None
# flag = 'ddp'
# flag = 'smddp'

if flag == 'ddp':
    distribution["mpi"]={"enabled": True}
    entry_point='pytorch_mnist_ddp.py'
elif flag == 'smddp':
    distribution["smdistributed"] = {"dataparallel": {"enabled": True}}
    entry_point='pytorch_mnist_smddp.py'
else:
    distribution = None
    entry_point='pytorch_mnist_sm.py'
distribution

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point=entry_point,
                    source_dir=source_dir,
                    role=role,
                    framework_version='2.0',
                    py_version='py310',
                    instance_count=instance_count,
                    instance_type=instance_type,
                    distribution=distribution,
                    metric_definitions=metric_definitions,
                    # profiler_config=profiler_config,
                    # rules=rules,
                    disable_profiler=True,
                    debugger_hook_config=False,
                    use_spot_instances=use_spot_instances,
                    max_wait=max_wait,
                    max_run=max_run,
                    hyperparameters={
                        'epochs': 5,
                                    },
                    sagemaker_session=sagemaker_session,
                    # tags=[{'Key':'Owner','Value':'username'}]
                   )

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [ ]:
from time import strftime
create_date = strftime("%m%d-%H%M%s")

experiment_name = 'mnist-exp1'
job_name=f'{experiment_name}-{create_date}'



estimator.fit(
    inputs={'training': s3_data_path}, 
    job_name=job_name,
    wait=False,
)

In [ ]:
sagemaker_session.logs_for_job(job_name=job_name, wait=True)